# 关于目标检测常用数据集那些事

In [1]:
import os
from xml.dom.minidom import parse,Document
from tqdm import tqdm
import json
import cv2

## 一.概述

在深度学习任务中，检测任务可谓是最令人头疼的，其中一个原因便是其数据集复杂的数据结构，对于图像级别的目标检测，常见的便有voc、coco两种格式，对于视频级别的目标检测（时空动作检测模型），常见的为ava数据集格式。本文以voc、coco数据集之间的相互转化为主，剖析如何完成这些数据集格式之间的转换。


## 二.几种格式的介绍

### 1.两种位置信息表示方式

首先需要明确，在2D检测任务中，如果想要标记一个物体就两种信息：**位置信息和类别信息**。其中，类别信息一般用**标记物体类别在classes_list(类别列表)中的下标**来表示。位置信息有两种表示方法：
- xyxy:
  $$
  (x_{min},y_{min},x_{max},y_{max})
  $$
- xywh:
  $$
  (x_{center},y_{center},w,h)
  $$

### 2.txt存储格式(包含buffer格式)
![图 2](images/d13b802d06319c89215a677ecba8db98c6dd1e6255dba000136465fb1387636c.png)  


现在，我们才来介绍一种非常“原始”的存储方式——yolo txt，顾名思义就是使用txt文件存储标签信息（包含了：类别下标、位置信息）。一般来说：存储格式要求如下：

- 每一行代表一个被标记的物体；
- 一行有5个数据，从左往右依次为：类别信息，四个位置信息
- 其中：类别信息最好**数字化**(转为下标形式)，位置信息需要**归一化**。

当然，yolo txt也可以在内存中表示，比如说使用python中的list数据表示为：
$$
[label\quad index,location\quad data]
$$

这种buffer中的格式，我们也经常称之为：**bbox数据格式**。

### 3.voc和xml


#### (1)voc概述

然后，我们来介绍一种现在使用可以说非常广泛的数据格式——VOC，全称为：Pascal Voc，至于它怎么产生和历史渊源咱们就不过多了解，重点了解其数据结构和组织形式。

对于刚刚接触目标检测的好homie一般都是先了解到VOC的，从官网上下载voc2007后，其文件结构为：

    └── VOCdevkit   #根目录
        └── VOC2007 #除了2007版本还有2012版本等
            ├── Annotations #存放xml文件，与JPEGImages中的图片对应，里面是标注物体信息以及图片信息等
            ├── ImageSets   #存放txt文件
            │   ├── Layout # 验证集和训练集的txt标记
            │   ├── Main # 不同类别物体的标记
            │   └── Segmentation # 验证集和训练集的txt标记
            ├── JPEGImages         #存放的是原生图片
            ├── SegmentationClass  #语义分割图片
            └── SegmentationObject #实例分割图片

其实没必要弄得很很复杂，对于今天我们搞定目标检测数据格式来说，我们只需要了解JPEGImages和Annotations足以。所以，一般来说你可以直接按照如下格式存储你自己标注的数据：

![图 1](images/15943038400967a71cbc22857b06df6b27f946e911757b406a27dcb33d1f910f.png)  

其中，JPEGImages存放你的数据集照片：

![图 2](images/29057aa29d70beca22d960d7d1aecd17a15fc1a2ce5289f6edfa748acd33004e.png)  


Annotations存放每张照片对应的信息文件(xml)：

![图 3](images/6c73d837f3373e90d194b5ecb780048e52d08243c9aec8852f31fb92502ef845.png)  
(可以看到xml文件和jpg图片文件是一一对应的(文件名字前缀一样))



那么这个xml文件是什么呢？如果学过多媒体或者web的同学，老师应该会给你们讲：像一些网站需要中小型的数据库来**存储一些数据**的时候，可以使用xml文件来代替，这是xml的一大功能，另外一个功能便是存储网页或者软件的**用户配置信息**。

#### (2)xml介绍

说白了，xml跟html、css、txt等文件一样，本质上都是存储数据的，只不过存储格式不一样、编码格式不一样以此来适应不同的使用场合罢了——那么xml的存储格式不同在哪儿呢？

html和xml是不是听起来很相似？没错，它俩可以说是一对兄弟，都是**树形结构**，有父、子、兄弟节点的，eg：

![图 4](images/93fefc2064e2ef97002908e4fb39d72a2e0581e500fbe97ccbf8aa2b64b64725.png)  

可以看到上图中，annotation是最大的节点，其次是有folder、filename等兄弟节点。那么我们想要的信息就存储在：object和size以及filename中。因此，如何把voc格式数据也就是xml文件转化为txt格式，关键就在如何把这三个信息读取出来。

### 4.coco格式

![图 1](images/4c9e3f515a2e5b6b843730ff14201d042737a8cfdcc981e159a7cfabcb614283.png)  


coco相较于voc数据集，共同点是都是可以运用在多个深度学习任务（目标检测、实例分割、语义分割等）中的数据集，其不同点：最为明显的就是COCO数据集所包含的种类数和图片数目更多。我们来看一下coco数据集的存放格式：

    ├─annotations_trainval2014
    │  └─annotations
    └─train2014
        └─train2014
其中train2014存放的就是原生图片，annotations下存放的有三种深度学习任务的对应的train和val标注信息文件。（6个）我们可以来看一下,这些json文件长什么样子：


In [4]:
test_path = r'F:\coco\annotations_trainval2014\annotations\instances_val2014.json'
file = open(test_path,'r')
obj = json.load(file)
print(type(obj))
print(obj.keys())
print(obj['info'])
print(obj['licenses'])

<class 'dict'>
dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])
{'description': 'COCO 2014 Dataset', 'url': 'http://cocodataset.org', 'version': '1.0', 'year': 2014, 'contributor': 'COCO Consortium', 'date_created': '2017/09/01'}
[{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nc/2.0/', 'id': 2, 'name': 'Attribution-NonCommercial License'}, {'url': 'http://creativecommons.org/licenses/by-nc-nd/2.0/', 'id': 3, 'name': 'Attribution-NonCommercial-NoDerivs License'}, {'url': 'http://creativecommons.org/licenses/by/2.0/', 'id': 4, 'name': 'Attribution License'}, {'url': 'http://creativecommons.org/licenses/by-sa/2.0/', 'id': 5, 'name': 'Attribution-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nd/2.0/', 'id': 6, 'name': 'Attribution-NoDerivs License'}, {'url': 'http://flickr.com/commons/usage/', 'id': 7, 'name': 'No kno

可以看到一个大字典，最重要的key就images、annotations、categories三种。一个大字典，下分3个主要key，其value均为list类型，三种key的每个list元素一一对应，元素又为一个小字典。

In [7]:
print(len(obj['images']))
print(type(obj['images']))
print(obj['images'][0])

40504
<class 'list'>
{'license': 3, 'file_name': 'COCO_val2014_000000391895.jpg', 'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000391895.jpg', 'height': 360, 'width': 640, 'date_captured': '2013-11-14 11:18:45', 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg', 'id': 391895}


images类中存放图片的下载url（我们自己写可以替换为自己存放的路径，或者直接不写），高、宽、文件名等主要信息，当然还一个最重要的——ID，我们自己写转换函数为了方便，可以直接用filename数字前缀作为ID。

In [8]:
print(len(obj['categories']))
print(type(obj['categories']))
print(obj['categories'][0])

80
<class 'list'>
{'supercategory': 'person', 'id': 1, 'name': 'person'}


哎，这儿有个很有意思的地方——这些类别不仅仅有**超类别**，还有子类别——估计是受到YOLOv2的wordtree启发？

In [9]:
print(len(obj['annotations']))
print(type(obj['annotations']))
print(obj['annotations'][0])

291875
<class 'list'>
{'segmentation': [[239.97, 260.24, 222.04, 270.49, 199.84, 253.41, 213.5, 227.79, 259.62, 200.46, 274.13, 202.17, 277.55, 210.71, 249.37, 253.41, 237.41, 264.51, 242.54, 261.95, 228.87, 271.34]], 'area': 2765.1486500000005, 'iscrowd': 0, 'image_id': 558840, 'bbox': [199.84, 200.46, 77.71, 70.88], 'category_id': 58, 'id': 156}


标注的话，可以看到有分割标注，但我们需要的是image_id和bbox标注，更重要的——可以看到291875>>40504，再加上这里只有一个bbox info，说明是一个标注物体为一个list元素。

另外，这里的位置信息表示方式还值得商榷，我目前估计大可能不是xyxy。经过查阅资料得知，bbox从左往右为左上角x坐标、左上角y坐标、宽度、长度。但是我们自己写完全可以和其他几个格式统一起来，或者按照自己所需。

## 三.转换实例

考虑到大家一般先接触到voc，同时coco和voc以及txt是外存上的数据，bbox是内存或者buffer中的数据，那么我们就以bbox作为**中心媒介数据格式**完成这三种数据集格式转换。

### 1.voc -> bbox

In [2]:
def readvocxml(xml_path, image_dir):
    """

    The function can read single xml file and transform information of xml file into a list containing:
    the filename of the xml indicates(str),
    the filepath of image that xml indicates(a str.you need to give the dir which this image located in.Aka,the second parameter.)
    the depth,height,width of the Image(three int data.channel first),
    the annotated objects' infomation.(
        a 2D int list:
        [
            row1:[label_1,xmin_1,ymin_1,xmax_1,ymax_1]
            row2:[label_2,xmin_2,ymin_2,xmax_2,ymax_2]
            ....
            row_i[label_i,xmin_i,ymin_i,xmax_i,ymax_i]
        ]
    )

    Args:

    xml_path:singal xml file's path.

    image_dir:the image's location dir that xml file indicates.
    (不加这个参数的话，就算不用我写的代码，都可能会出些很恶心的小bug，建议和JPEGImages联立起来)


    """
    tree = parse(xml_path)
    rootnode = tree.documentElement
    sizenode = rootnode.getElementsByTagName('size')[0]
    width = int(sizenode.getElementsByTagName('width')[0].childNodes[0].data)
    height = int(sizenode.getElementsByTagName('height')[0].childNodes[0].data)
    depth = int(sizenode.getElementsByTagName('depth')[0].childNodes[0].data)

    name_node = rootnode.getElementsByTagName('filename')[0]
    filename = name_node.childNodes[0].data

    path = image_dir + '\\' + filename

    objects = rootnode.getElementsByTagName('object')
    objects_info = []
    for object in objects:
        label = object.getElementsByTagName('name')[0].childNodes[0].data
        xmin = int(object.getElementsByTagName('xmin')[0].childNodes[0].data)
        ymin = int(object.getElementsByTagName('ymin')[0].childNodes[0].data)
        xmax = int(object.getElementsByTagName('xmax')[0].childNodes[0].data)
        ymax = int(object.getElementsByTagName('ymax')[0].childNodes[0].data)
        info = []
        info.append(label)
        info.append(xmin)
        info.append(ymin)
        info.append(xmax)
        info.append(ymax)
        objects_info.append(info)

    return [filename, path, depth, height, width, objects_info]

In [3]:
objects_info = readvocxml(xml_path=r'./voc/Annotations/000005.xml',image_dir=r'D:\PythonCode\OD_dataset_format\voc\JPEGImages')
print(objects_info) 

['000005.jpg', 'D:\\PythonCode\\OD_dataset_format\\voc\\JPEGImages\\000005.jpg', 3, 375, 500, [['chair', 263, 211, 324, 339], ['chair', 165, 264, 253, 372], ['chair', 5, 244, 67, 374], ['chair', 241, 194, 295, 299], ['chair', 277, 186, 312, 220]]]


一般来说，这里的readxml已经足够应付普通的目标检测了，但一般我自己为了应付使用DIF的情况并使得各个数据之间的转换更加方便，会多读取几个参数，反正根据自己所需改代码吧。

### 2.bbox -> txt

In [9]:
def bbox2txt(objects_info,filename,save_dir):
    """
    Args:
        objects_info:（已经是归一化和标签数字化的2D列表）
        a 2D int list:
            [
                row1:[label_1,xmin_1,ymin_1,xmax_1,ymax_1]
                row2:[label_2,xmin_2,ymin_2,xmax_2,ymax_2]
                ....
                row_i[label_i,xmin_i,ymin_i,xmax_i,ymax_i]
            ]
        filename:the txt filename.（txt前缀）
        save_dir:the directory you want to save txt file.
    """
    with open("{}/{}.txt".format(save_dir,filename),'w') as f:
        lines = ""
        for object_ in objects_info:
            line = "{} {} {} {} {}\n".format(
                object_[0],
                object_[1],
                object_[2],
                object_[3],
                object_[4]
            )
            lines = lines + line
        f.write(lines)

In [12]:
# 批量转换
ann_dir = r'D:\PythonCode\OD_dataset_format\voc\Annotations'
txt_save_dir = r'D:\PythonCode\OD_dataset_format\txt'
txt_path=r'D:\PythonCode\OD_dataset_format\voc\classes.txt'
for ann_name in tqdm(os.listdir(ann_dir)):
    ann_path = ann_dir + '/' + ann_name 
    filename, path, depth, height, width, objects_info = readvocxml(ann_path,txt_path)   # voc -> bbox
    filename = filename.split('.')[0]
    bbox2txt(objects_info,filename,txt_save_dir)

100%|██████████| 197/197 [00:00<00:00, 208.86it/s]


![图 5](images/8c6967044ed1fcd4374ddc2897378af1d7692b9cb62a3cf92d7a665e4677df3e.png)  


### 3.txt -> bbox

In [13]:
def txt2bbox(txt_path):
    """
    Args:
        txt_path:单个txt路径
    Returns:
        objects_info:与txt存储方式一样的2D列表
        filename:文件名前缀
    """
    objects_info = []
    sig = 0 # 0:filename前是/,  1:filename前是\\
    for i in range(1,len(txt_path)+1):
        if txt_path[-i]=='\\':
            sig=1
            break
        if txt_path[-i]=='/':
            break
    if sig:
        filename = txt_path.split('\\')[-1].split('.')[0]
    else:
        filename = txt_path.split('/')[-1].split('.')[0]

    with open(txt_path,'r') as f:
        info_list = f.read().split('\n')[:-1]
        info_list = [i.split() for i in info_list]
 
        for info in info_list:
            info_ = []
            info_.append(info[0])
            info_.append(int(info[1]))    # 位置信息: str->float
            info_.append(int(info[2]))
            info_.append(int(info[3]))
            info_.append(int(info[4]))
            objects_info.append(info_) 
    return objects_info,filename

In [14]:
objects_info,filename = txt2bbox(r"D:\PythonCode\OD_dataset_format\txt\000005.txt")
print(objects_info, filename)

[['chair', 263, 211, 324, 339], ['chair', 165, 264, 253, 372], ['chair', 5, 244, 67, 374], ['chair', 241, 194, 295, 299], ['chair', 277, 186, 312, 220]] 000005


### 4.bbox -> xml

这里要稍微麻烦一些，我们需要了解如何写一个”xml树“。

![图 6](images/3cad3e6ef0a4ac864b97556f2de06d8e78654e22380c709cafedab92a774f0ac.png)  


为了简化使用，我们只取上图中框出来的信息，其他参数可以根据自己所需加入。(可能存在多个object)

In [4]:
def bbox2xml(objects_info, img_filename, image_shape, save_dir):
    """
    Args:
        objects_info:归一化的、标签数字化的2D列表
        img_filename:xml对应图片文件名
        image_shape:为了方便使用者自定义，需要手动输入
        classes_path:classes txt存储路径
        save_dir:xml保存目录
    """
    d,h,w = image_shape
    doc = Document()
    # 文件根节点
    annotation = doc.createElement('annotation')
    doc.appendChild(annotation)
    # 建立四个仅次于文件根节点的兄弟节点
    filename_node = doc.createElement('filename')
    annotation.appendChild(filename_node)
    filename_ = doc.createTextNode(img_filename)
    filename_node.appendChild(filename_)
    size_node = doc.createElement('size')
    annotation.appendChild(size_node)    
    # size节点下分三个节点:width,height,depth
    w_node = doc.createElement('width')
    h_node = doc.createElement('height')
    d_node = doc.createElement('depth')
    size_node.appendChild(w_node)
    size_node.appendChild(h_node)
    size_node.appendChild(d_node)
    w_ = doc.createTextNode(str(w))
    h_ = doc.createTextNode(str(h))
    d_ = doc.createTextNode(str(d))
    w_node.appendChild(w_)
    h_node.appendChild(h_)
    d_node.appendChild(d_)
    
    for i in range(len(objects_info)):
        object_ = objects_info[i]
        name,xmin,ymin,xmax,ymax = object_
        object_node = doc.createElement('object')
        annotation.appendChild(object_node)
        # object下分两个节点：name和bndbox
        name_node = doc.createElement('name')
        object_node.appendChild(name_node)
        name_ = doc.createTextNode(name)
        name_node.appendChild(name_)
        # 位置信息结点建立
        bnd_node = doc.createElement('bndbox')
        object_node.appendChild(bnd_node)
        xmin_node = doc.createElement('xmin')
        ymin_node = doc.createElement('ymin')
        xmax_node = doc.createElement('xmax')
        ymax_node = doc.createElement('ymax')
        bnd_node.appendChild(xmin_node)
        bnd_node.appendChild(ymin_node)
        bnd_node.appendChild(xmax_node)
        bnd_node.appendChild(ymax_node)
        xmin_ = doc.createTextNode(str(xmin))
        ymin_ = doc.createTextNode(str(ymin))
        xmax_ = doc.createTextNode(str(xmax))
        ymax_ = doc.createTextNode(str(ymax))
        xmin_node.appendChild(xmin_)
        ymin_node.appendChild(ymin_)
        xmax_node.appendChild(xmax_)
        ymax_node.appendChild(ymax_)
    
    # 写入文件
    with open(save_dir+'/'+img_filename.split('.')[0]+'.xml','w') as f:
        f.write(doc.toprettyxml(indent="    "))    

In [11]:
objects_info,filename = txt2bbox(r"D:\PythonCode\OD_dataset_format\txt\000005.txt")
bbox2xml(objects_info,filename+'.jpg',(3,375,500), './')

![图 7](images/b3e97dee7ed8835684ad85589aae00f07ccef1a69fde849a9e5ffdd2d0b34eec.png)  

对比一下，发现没有问题，说明功能实现成功。

### 5.voc -> coco

这里为什么不直接bbox转为coco，是因为，coco格式数据集其实是整体的——一整个数据集标注就用一个json文件存储，而且最重要的是图片与标注关联度很强，那么直接读入bbox来写不太容易，传入参数太多。

此数据转换的难点在于如何写入json文件，其实很简单，前面已经演示了，python可以将json文件解析为自己的字典类型。还记得刚刚讲的三个主要keys吗？

- images:图片名字、路径、c、h、w、id(可选，直接写成数字前缀也行)；
- annotations:bbox信息、对应图片id；
- categories:类别id、类别名字；



In [22]:
def voc2coco(jpg_dir,ann_dir,txt_path,json_path):
    """
        Args:
            jpg_dir:JPEGimage路径
            ann_dir:Annotations路径
            txt_path:classes.txt路径
            json_path:存储的json文件路径
    """
    obj = {'images':[],'annotations':[],'categories':[]}
    # 先把种类搞定
    with open(txt_path,'r') as f:
        cls_list = f.read().split('\n')[:-1]
    for i in range(len(cls_list)):
        cate_dict = {}
        cate_dict['id']=i   # clslist下标作为cate_id
        cate_dict['name']=cls_list[i]   
        obj['categories'].append(cate_dict)
    # 搞定images和annotations
    for filename in tqdm(os.listdir(ann_dir)):
        img_dict = {}
        xml_path = ann_dir + '/' + filename
        filename, path, depth, height, width, objects_info = readvocxml(xml_path,jpg_dir)
        # img_dict
        img_dict['filename']=filename
        img_dict['id']=filename.split('.')[0]   # filename前缀作为id
        img_dict['depth']=depth
        img_dict['height']=height
        img_dict['width']=width
        img_dict['path']=path
        obj['images'].append(img_dict)
        # ann_duct
        for object_ in objects_info:
            ann_dict = {}
            ann_dict['bbox']=object_[1:]
            ann_dict['category_id']=object_[0]
            ann_dict['image_id']=filename.split('.')[0]
            obj['annotations'].append(ann_dict)
    # 最后写入json文件
    with open(json_path,'w') as fb:
        json.dump(obj,fb,indent="   ")

In [23]:
voc2coco(
    r'D:\PythonCode\OD_dataset_format\voc\JPEGImages',
    r'D:\PythonCode\OD_dataset_format\voc\Annotations',
    r'D:\PythonCode\OD_dataset_format\voc\classes.txt',
    r'D:\PythonCode\OD_dataset_format\coco\annotation_train\voc2007tococo.json'
)

100%|██████████| 197/197 [00:00<00:00, 1683.38it/s]


### 6.coco -> voc

In [26]:
def coco2voc(src_dir,json_path,ann_dir,txt_path):
    """
        Args:
            src_dir:coco图片目录
            json_path:coco标注json文件路径
            ann_dir:voc的Annotations目录
            txt_path:为voc生成的classes.txt路径
    """
    with open(json_path,'r') as f:
        obj = json.load(f)
        ann_info = obj['annotations']
        cls_info = obj['categories']
        # 先生成classes.txt:
        id_cate = {}
        for item in cls_info:
            id_cate[int(item['id'])]=item['name']
        cls_list = [0 for i in range(len(id_cate))]
        for key in id_cate.keys():
            cls_list[key]=id_cate[key]  # 要保证id从小到大输出
        content = ""
        for cls in cls_list:
            content = content + cls + '\n'
        with open(txt_path,'w') as f:
            f.write(content)        
        # 建立图片-标注物体字典并完成转换
        img_obj_dict = {}
        for item in ann_info:
            img_obj_dict[item['image_id']]=[]
        for item in ann_info:
            img_obj_dict[item['image_id']].append([item['category_id']]+item['bbox'])
        for key in tqdm(img_obj_dict.keys()):
        ######################  tips:默认图片格式为jpg，如果不是要改一下下方{}后的".jpg"  ###################### 
            img_path = src_dir + '/' + "{}.jpg".format(key)
            shape = cv2.imread(img_path).shape
            objects_info = img_obj_dict[key]
            bbox2xml(objects_info,key+'.jpg',(shape[-1],shape[0],shape[1]),ann_dir)
        ######################################################################################################

In [27]:
coco2voc(
    r'D:\PythonCode\OD_dataset_format\coco\train2014',
    r'D:\PythonCode\OD_dataset_format\coco\annotation_train\voc2007tococo.json',
    r'D:\PythonCode\OD_dataset_format\voc(coco-trans)\Annotations',
    r'D:\PythonCode\OD_dataset_format\voc(coco-trans)\classes.txt'
)

100%|██████████| 197/197 [00:01<00:00, 105.21it/s]


## 四.总结

![图 1](images/ec4219ebba8a84a237df80029727c99e1c3e35754f9c36ec4477617ec11069e0.png)  
